In [ ]:
//Oscar Delgado and Colin Murphy LISP projects 

In [ ]:
import scala.util.control.Exception.allCatch

In [ ]:
// TYPES

sealed trait List[+A]
case object Empty extends List[Nothing]
case class Cons[A](x : A, xs : List[A]) extends List[A]

sealed trait Pair[+A,+B]
case class MkPair[A,B](fst : A, snd : B) extends Pair[A,B]

sealed trait Symbols
case object EmptyS extends Symbols 
case class Extend(k : Expr, v : Expr, env : Symbols) extends Symbols

sealed trait Maybe[+A]
case object Nothing extends Maybe[Nothing]
case class Just[A](fromJust : A) extends Maybe[A]

//sealed trait Procedures

sealed trait Expr
case class LispNum(x : scala.Int) extends Expr
case class LispBin(x : Boolean) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Div(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : Expr, x : Expr, y : Expr) extends Expr
case class FunDef(id: Expr, x : Expr) extends Expr
case class FunCall(id : Expr, x : Expr) extends Expr
case class Procedure(id : Expr, e : Expr, pi : Symbols) extends Expr
case object Error extends Expr
case object OpenP extends Expr
case object CloseP extends Expr
case object holder extends Expr
case class Define(id : Expr, x : Expr) extends Expr

In [ ]:
// Useful Function

def lookup(dict : Symbols, k : Expr) : Maybe[Expr] = dict match {
    case EmptyS        => Nothing
    case Extend(k1, v, xs) => k == k1 match {
        case true  => Just(v)
        case false => lookup(xs, k)
    }
}

def pow(x : Int , y : Int) : Int = y match{
    case 0 => 1
    case 1 => x
    case y => x * (pow(x,y-1))
}

var GStore : Symbols = EmptyS


In [ ]:
//Evaluation

def bin_Arith(t : Char, v1 : Expr, v2 : Expr) : Expr = (t,v1,v2) match{
    case ('+',LispNum(v1),LispNum(v2)) => LispNum(v1+v2) 
    case ('-',LispNum(v1),LispNum(v2)) => LispNum(v1-v2) 
    case ('*',LispNum(v1),LispNum(v2)) => LispNum(v1*v2) 
    case ('/',LispNum(v1),LispNum(v2)) => LispNum(v1/v2) 
    case _ => Error
}
def bool_Arith(b : Char, v1 : Expr, v2 : Expr) : Expr = (b,v1,v2) match{
    case ('&',LispBin(v1),LispBin(v2)) => LispBin(v1 && v2) 
    case ('|',LispBin(v1),LispBin(v2)) => LispBin(v1 || v2) 
    case _ => Error
}


def Eval(store : Symbols, expr : Expr) : Expr = expr match{
    case LispNum(n) => LispNum(n)
    case LispBin(n) => LispBin(n)
    case Ident(x) => lookup(store , Ident(x)) match{
        case Just(v) => v
        case _ => Error
    }
    case Plus(x,y) => bin_Arith('+' , Eval(store,x),Eval(store,y))
    case Minus(x,y) => bin_Arith('-' , Eval(store,x),Eval(store,y))
    case Mult(x,y) => bin_Arith('*' , Eval(store,x),Eval(store,y))
    case Div(x,y) => bin_Arith('/' , Eval(store,x),Eval(store,y))
    case Pow(x,y) => (x,Eval(store, y)) match{
        case (x,LispNum(z)) => (x,z >= 0) match{
            case (LispNum(x),true) => LispNum(pow(x,z))
            case _ => Error
        }
        case _ => Error
    }
    case Eq(x,y) => Eval(store,x) == Eval(store,y) match{
        case x => LispBin(x)
    }
    case And(x,y) => bool_Arith('&' , Eval(store,x),Eval(store,y))
    case Or(x,y) => bool_Arith('|' , Eval(store,x),Eval(store,y))
    case Neg(x) => Eval(store,x) match{
        case LispNum(x) => LispNum(-x)
        case LispBin(x) => LispBin(!x)
        case _ => Error
    }  
    case IfThenElse(p, e_t, e_f) => Eval(store, p) match{
        case LispBin(true)  => Eval(store, e_t)
        case LispBin(false) => Eval(store, e_f)
        case _             => Error
    }
    case Let(x,y,z) => Eval(Extend(x,Eval(store,y),store),z)
    case FunDef(x , e1) => Procedure(x,e1,store)
    case FunCall(e1 , e2) => Eval(store,e1) match{
        case Procedure(p,e3,pi)=> Eval(store,e2) match{
            case x => {
            val new_env = Extend(p, x, pi)
            Eval(new_env , e3)
            }
        }
        case _ => Error
    }
    case Procedure(x,y,z) => Procedure(x,y,z)
    case Define(x,y) => Define(x,y)
    case Error=>Error
    
}

In [ ]:
// String Splitting to Array[String] to Array[Expr]

def ItoS(input : String) : Array[String] = input match{
    case input => input.split(" ")
}

def isNumber(s: String): Boolean = (allCatch opt s.toDouble).isDefined

def makeArrayE(stringA : Array[String]) : Array[Expr] = {
    var exprA : Array[Expr] = new Array[Expr](stringA.length)
    var i = 0;
    while(i < stringA.length){
        if(stringA.apply(i) == "("){
            exprA(i) = OpenP;

        }
        else if(stringA.apply(i) == ")"){
            exprA(i) = CloseP;

        }else if(stringA.apply(i) == "+"){
            exprA(i) = Plus(holder,holder);

        }else if(stringA.apply(i) == "-"){
            exprA(i) = Minus(holder,holder);

        }else if(stringA.apply(i) == "*"){
            exprA(i) = Mult(holder,holder);

        }else if(stringA.apply(i) == "/"){
            exprA(i) = Div(holder,holder);

        }else if(stringA.apply(i) == "="){
            exprA(i) = Eq(holder,holder);

        }else if(stringA.apply(i) == "true"){
            exprA(i) = LispBin(true);

        }else if(stringA.apply(i) == "false"){
            exprA(i) = LispBin(false);

        }else if(isNumber(stringA.apply(i)) == true){
            exprA(i) = LispNum(stringA.apply(i).toInt);

        }else if(stringA.apply(i) == "&&"){
            exprA(i) = And(holder,holder); 

        }else if(stringA.apply(i) == "||"){
            exprA(i) = Or(holder,holder);

        }else if(stringA.apply(i) == "!"){
            exprA(i) = Neg(holder);

        }else if(stringA.apply(i) == "x"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "y"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "z"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "f"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "g"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "h"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "if"){
            exprA(i) = IfThenElse(holder,holder,holder);

        }else if(stringA.apply(i) == "let"){
          exprA(i) = Let(holder,holder,holder);

        }else if(stringA.apply(i) == "define"){
          exprA(i) = Define(holder,holder);
            
        }else if(stringA.apply(i) == "fd"){
          exprA(i) = FunDef(holder,holder);
            
        }else if(stringA.apply(i) == "fc"){
          exprA(i) = FunCall(holder,holder);
            
        }else if(stringA.apply(i) == "^"){
          exprA(i) = Pow(holder,holder);
            
        }else if(stringA.apply(i) == "p"){
          exprA(i) = Procedure(holder,holder,GStore);
            
        }
        else{
             exprA(i) = Error;
        }
        i += 1;
    }
    return exprA
}


In [ ]:
//Array[Expr] => Expr 

var i = 0

def AtoE(A : Array[Expr]) : Expr = A {
    i+=1
    if (A.apply(i) == OpenP){
        return AtoE(A)
    }
    else if (A.apply(i) == Plus(holder,holder)){
            var lol = Plus(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
        }
    else if (A.apply(i) == Minus(holder,holder)){
         var lol = Minus(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Mult(holder,holder)){
       var lol = Mult(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Div(holder,holder)){
       var lol = Div(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Eq(holder,holder)){
        var lol = Eq(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == And(holder,holder)){
        var lol = And(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Or(holder,holder)){
        var lol = Or(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == IfThenElse(holder,holder,holder)){
        var lol = IfThenElse(AtoE(A),AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Let(holder,holder,holder)){
        var lol = Let(AtoE(A),AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == FunDef(holder,holder)){
        var lol = FunDef(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == FunCall(holder,holder)){
        var lol = FunCall(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Neg(holder)){
        var lol = Neg(AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Pow(holder,holder)){
        var lol = Pow(AtoE(A),AtoE(A))
            i+=1
            if (i == A.length-1){i=0}
            return lol
    }
    else if (A.apply(i) == Define(holder,holder)){
        var t1 = AtoE(A)
        var t2 = AtoE(A)
        GStore = Extend(t1 , t2,GStore)
        i+=1
            if (i == A.length-1){i=0}
        return Define(t1,t2)
    }
    if (A.apply(i) == Procedure(holder,holder,GStore)){
        var lol = Procedure(AtoE(A),AtoE(A),GStore)
        i+=1
            if (i == A.length-1){i=0}
        return lol
        }
    else if (A.apply(i) == CloseP){
        if (i == A.length-1){i=0}
        AtoE(A)
    }
    else {
        if (i == A.length-1){i=0}
        return  A.apply(i)
            
            
        }
    return Error

}



In [ ]:
var example0 =AtoE(makeArrayE(ItoS("( ! true )"))) 
Eval(GStore,example0)

In [ ]:
var example1 = AtoE(makeArrayE(ItoS("( if ( = true true ) 42 0 )"))) 
Eval(GStore,example1)

In [ ]:
var example2 = AtoE(makeArrayE(ItoS("( if ( = true false ) 42 ( ^ 3 3 ) )"))) 
Eval(GStore,example2)

In [ ]:
AtoE(makeArrayE(ItoS("( define x 3 )"))) 

In [ ]:
var example3 = AtoE(makeArrayE(ItoS("( * x ( + x 2 ) )")))
Eval(GStore,example3)

In [ ]:
var example4 = AtoE(makeArrayE(ItoS("( let x 5 ( * x 12 ) )")))
Eval(GStore,example4)

In [ ]:
AtoE(makeArrayE(ItoS("( define f ( p x ( + x 10 ) ) )")))

In [ ]:
var example5 = AtoE(makeArrayE(ItoS("( fc f 5 )")))
Eval(GStore , example5)

In [ ]:
var example6 = AtoE(makeArrayE(ItoS("( let g ( fd y ( - y 25 ) ) ( fc g 100 ) )")))
Eval(GStore,example6)